The goal of this notebook is to join all of the different datasets into one giant matrix which can then be fed into the autoencoder. To start, let's look at each dataset that we have.

In [ ]:
%%bash
head -n 10 ng00061/WES_release3AtlasOnly_vep80_most_severe_consequence_per_gene.txt

In [ ]:
%%bash
# These have different numbers of columns of course. Need to fix this maybe? Will only use the coordinates from this
# for now, this is the primary key to link all the other datasets.
head -n 10 ng00061/WES_release3AtlasOnly_rolling_flat_annotation.txt
cut -f1-5 WGS_v1_rolling_flat_annotation.txt > WGS_v1_rolling_flat_annotation.pos_only.txt

In [ ]:
%%bash
head -n 10 igap/IGAP_stage_1_2_combined.txt

In [ ]:
%%bash
# I will join all of the chromosomes into one file.
cat ng00039/pvalue_only/metaanalysis/pvalueonly_METAANALYSIS1_chr*.TBL | perl -pe 's/^(\d+)-(\d+)/$1\t$2/g' > pvalue_only.tsv
head -n 10 pvalue_only.tsv

In [ ]:
%%bash
#I'm not sure what the difference is between these but I am assuming we get the inverse of controls here.
# We will use the controls for a confusion matrix
ls mayo_egwas/
head mayo_egwas/Hap300_CER_All.txt

In [ ]:
import pandas as pd